# Data Analysis and Cleaning of MatchedBeer Dataset

The MatchedBeer dataset combines data from BeerAdvocate and RateBeer, two of the most prominent online platforms for beer ratings. This combined dataset was created to address the herding effect in online ratings, where early ratings can skew subsequent user opinions. By merging the two platforms, the MatchedBeer dataset allows for the study of the same beers rated independently on each platform, thereby providing a unique opportunity to analyze unbiased rating patterns.

## Matching Process Overview
The matching process is described in detail in the paper ["When Sheep Shop: Measuring Herding Effects in Product Ratings with Natural Experiments"](https://dlab.epfl.ch/people/west/pub/Lederrey-West_WWW-18.pdf), focusing on accurately aligning beers from both websites. The matching is performed in two phases to prioritize precision:

1. **Brewery Alignment**: Breweries across BeerAdvocate and RateBeer are aligned based on their exact location (state or country), with names represented as TF-IDF vectors to compute cosine similarity.
2. **Beer Alignment**: Within matched breweries, beers are aligned based on name similarity and require an exact match in alcohol by volume (ABV). Before matching, any shared tokens in brewery and beer names are removed to improve accuracy.

The algorithm ensures high precision by setting strict thresholds for cosine similarity and enforcing a significant gap between the best and second-best matches. This approach results in a dataset where brewery matching has a reported precision of 99.6%, and beer matching achieves a precision of 100%, meaning that only highly confident matches are included, though some potential matches may be excluded.

## Structure of MatchedBeer Dataset
The dataset contains essential information on beers, breweries, users, and user ratings from both platforms. Since the user bases of BeerAdvocate and RateBeer differ geographically, the combined dataset reflects a more balanced view. BeerAdvocate is more U.S.-centric, while RateBeer has a more internationally diverse user base. Despite these differences, the matched dataset has been verified for both internal and external validity, making it well-suited for analyzing the influence of initial ratings on user perceptions across platforms.

MatchedBeer provides insights into how early ratings impact subsequent ones by comparing ratings for the same beer on both sites, especially when early ratings on each platform differ significantly. By standardizing and aligning ratings across time and platforms, MatchedBeer reveals patterns that allow for the identification of herding effects, helping researchers understand the biases in user-generated ratings.

In [5]:
# Import all the necessary libraries
import polars as pl
import tqdm
import os

# Import the file in the utils folder
import sys
sys.path.append('../../utils')
from data_desc import remove_whitespaces, describe_numbers, describe

# Define the paths
SRC_DATA_PATH = '../../../data/MatchedBeer'

#### Data Exploration

In [6]:
# This function loads and processes the matched_beer fioles so that all of the columns are appended with _ba or _rb depending on which dataset they are from
def load_and_process_files(file_paths):
    processed_dataframes = []
    
    for file_path in file_paths:
        # Load the first row to get the data source identifiers (e.g., "ba", "ba_1", "rb", etc.)
        header_row = pl.read_csv(file_path, n_rows=1, has_header=False)
        column_source = header_row.row(0)
        
        # Clean the source identifiers to only keep "ba" or "rb" without any suffixes
        source_identifiers = [source.split('_')[0] for source in column_source]
        
        # Load the actual data, skipping the first row
        df = pl.read_csv(file_path, skip_rows=1)
        
        # Rename columns based on the cleaned source identifiers
        new_column_names = [
            f"{name}_{source}".replace("_duplicated_0", "") for name, source in zip(df.columns, source_identifiers)
        ]
        df = df.rename({old: new for old, new in zip(df.columns, new_column_names)})
        
        # Append the processed DataFrame to the list
        processed_dataframes.append(df)
    
    return processed_dataframes

In [7]:
file_paths = [SRC_DATA_PATH + "/beers.csv", SRC_DATA_PATH + "/breweries.csv", SRC_DATA_PATH + "/ratings.csv", SRC_DATA_PATH + "/users.csv"]
dataframes_list = load_and_process_files(file_paths)

df_beers = dataframes_list[0]
df_breweries = dataframes_list[1]
df_ratings = dataframes_list[2]
df_users = dataframes_list[3]

##### Beers dataset
We'll start by looking at the beers dataset.

In [8]:
df_beers.sample(5)

abv_ba,avg_ba,avg_computed_ba,avg_matched_valid_ratings_ba,ba_score_ba,beer_id_ba,beer_name_ba,beer_wout_brewery_name_ba,brewery_id_ba,brewery_name_ba,bros_score_ba,nbr_matched_valid_ratings_ba,nbr_ratings_ba,nbr_reviews_ba,style_ba,zscore_ba,abv_rb,avg_rb,avg_computed_rb,avg_matched_valid_ratings_rb,beer_id_rb,beer_name_rb,beer_wout_brewery_name_rb,brewery_id_rb,brewery_name_rb,nbr_matched_valid_ratings_rb,nbr_ratings_rb,overall_score_rb,style_rb,style_score_rb,zscore_rb,diff_scores,sim_scores
f64,f64,f64,f64,f64,i64,str,str,i64,str,f64,i64,i64,i64,str,f64,f64,f64,f64,f64,i64,str,str,i64,str,i64,i64,f64,str,f64,f64,f64,f64
5.3,3.3,3.294545,3.3,82.0,118743,"""1814 Brown Ale""","""1814 Brown Ale""",31492,"""Shebeen Brewing Company""",null,3,11,3,"""American Brown Ale""",-1.271668,5.3,2.64,2.0,2.0,282680,"""Shebeen 1814 Colonial Brown Al…","""1814 Ale Colonial Brown""",16166,"""Shebeen Brewing Company""",1,1,null,"""Brown Ale""",null,-2.511123,0.615975,0.806305
4.5,3.91,3.746923,3.6425,85.0,37060,"""Bramling Cross""","""Cross Bramling""",8203,"""Paddock Wood Brewing Co.""",null,12,13,12,"""English Bitter""",-0.135749,4.5,2.85,3.15,3.15,146555,"""Paddock Wood Bramling Cross""","""Cross Bramling""",5562,"""Paddock Wood Microbrewery""",2,2,null,"""Bitter""",null,-0.360609,1.0,1.0
6.9,3.77,3.77,null,null,219537,"""V22""","""V22""",34426,"""Jailbreak Brewing Company""",null,0,2,0,"""American IPA""",-0.445024,6.9,null,null,null,407613,"""Jailbreak V22""","""V22""",19493,"""Jailbreak Brewing Company""",0,0,null,"""India Pale Ale (IPA)""",null,null,1.0,1.0
4.5,4.21,4.21,null,null,264175,"""Cookies & Cream""","""Cream Cookies""",34063,"""Edmund's Oast""",null,0,1,0,"""American Stout""",0.460288,4.5,3.26,3.9,3.9,486713,"""Edmunds Oast Cookies & Cream""","""Cream Cookies""",18935,"""Edmunds Oast""",2,2,null,"""Sweet Stout""",null,0.752968,1.0,1.0
5.0,null,null,null,null,177129,"""Mango Mae Hefeweizen""","""Mango Mae Hefeweizen""",40904,"""PerroVida Craft Beer""",null,0,0,0,"""Hefeweizen""",null,5.0,null,null,null,278907,"""PerroVida Mango Mae Hefeweizen""","""Mango Mae Hefeweizen""",18758,"""PerroVida Craft Beer""",0,0,null,"""German Hefeweizen""",null,null,1.0,1.0


This dataset has the following structure

| Column Name | Description |
|-------------|-------------|
| `beer_id_ba` / `beer_id_rb` | Unique identifier for each beer in BA/RB | 
| `beer_name_ba` / `beer_name_rb` | Name of the beer in BA/RB |
| `beer_wout_brewery_name_ba` / `beer_wout_brewery_name_rb` | Beer name without brewery name in BA/RB |
| `brewery_id_ba` / `brewery_id_rb` | Unique identifier for the brewery in BA/RB |
| `brewery_name_ba` / `brewery_name_rb` | Name of the brewery in BA/RB |
| `style_ba` / `style_rb` | Beer style in BA/RB |
| `abv_ba` / `abv_rb` | Alcohol By Volume percentage in BA/RB |
| `nbr_ratings_ba` / `nbr_ratings_rb` | Number of ratings in BA/RB |
| `nbr_reviews_ba` | Number of reviews in BA |
| `avg_ba` / `avg_rb` | Average rating in BA/RB |
| `avg_computed_ba` / `avg_computed_rb` | Computed average rating in BA/RB |
| `zscore_ba` / `zscore_rb` | Standardized score in BA/RB |
| `nbr_matched_valid_ratings_ba` / `nbr_matched_valid_ratings_rb` | Number of matched valid ratings in BA/RB |
| `avg_matched_valid_ratings_ba` / `avg_matched_valid_ratings_rb` | Average of matched valid ratings in BA/RB |
| `ba_score_ba` | BeerAdvocate score |
| `bros_score_ba` | Bros score in BA |
| `overall_score_rb` | Overall score in RB |
| `style_score_rb` | Style-specific score in RB |
| `diff_scores` | Difference in scores between BA and RB |
| `sim_scores` | Similarity in scores between BA and RB |

Now that we have an idea what the structure of our datasets looks like, let's see if our dataset is complete or whether it contains a lot of missing entries.

In [9]:
describe(df_beers)

+------------------------------+---------+------------------+---------------+-----------+-----------------------+-------------------+
| Column                       | Type    |   Not null count |   Nulls count |   Nulls % |   Unique values count |   Unique values % |
|------------------------------+---------+------------------+---------------+-----------+-----------------------+-------------------|
| abv_ba                       | Float64 |            45640 |             0 |    0.00 % |                   351 |            0.77 % |
| avg_ba                       | Float64 |            40285 |          5355 |   11.73 % |                   373 |            0.93 % |
| avg_computed_ba              | Float64 |            40285 |          5355 |   11.73 % |                 12090 |           30.01 % |
| avg_matched_valid_ratings_ba | Float64 |            28272 |         17368 |   38.05 % |                  6658 |           23.55 % |
| ba_score_ba                  | Float64 |            10499 | 

In [10]:
describe_numbers(df_beers, filters =["beer_id_ba", "brewery_id_ba", "beer_id_rb", "brewery_id_rb"])

+------------------------------+-----------+-----------+-----------+------------+-----------+----------+---------+
| Column                       |      Mean |       Std |       25% |        50% |       75% |      Min |     Max |
|------------------------------+-----------+-----------+-----------+------------+-----------+----------+---------|
| abv_ba                       |   6.32086 |   1.85773 |         5 |          6 |       7.2 |     0.38 |    67.5 |
| avg_ba                       |   3.73237 |  0.439988 |      3.52 |       3.78 |         4 |        1 |       5 |
| avg_computed_ba              |   3.72785 |  0.424824 |      3.52 |     3.7725 |         4 |        1 |       5 |
| avg_matched_valid_ratings_ba |   3.74777 |  0.484775 |      3.52 |    3.80833 |     4.045 |        1 |       5 |
| ba_score_ba                  |    84.871 |   3.28287 |        83 |         85 |        86 |       63 |     100 |
| bros_score_ba                |   86.1623 |    8.5394 |        83 |         88 

1. Alcohol By Volume (ABV)
   - The `abv_ba` and `abv_rb` columns show identical distributions across BeerAdvocate and RateBeer, with a mean ABV of approximately 6.32%.
   - This similarity suggests a good alignment in the types of beers reviewed on both platforms. The maximum ABV of 67.5% is possibly an outlier, likely representing rare extreme beers, as typical ABVs are much lower.

2. Average Ratings (BeerAdvocate vs. RateBeer)
   - The `avg_ba` (BeerAdvocate) mean is 3.73, while `avg_rb` (RateBeer) is significantly lower at 3.14.
   - This aligns with previous findings that BeerAdvocate users tend to rate beers more generously than RateBeer users. The higher `std` value for `avg_ba` also indicates a wider range in BeerAdvocate’s ratings, suggesting more variability in user opinions.

3. Computed Averages (Consistency in Ratings)
   - `avg_computed_ba` and `avg_computed_rb` refer to recalculated averages, showing slightly lower standard deviations than their direct counterparts (`avg_ba` and `avg_rb`), indicating a slight smoothing effect.
   - Both BeerAdvocate and RateBeer have computed averages (`avg_computed_ba` vs. `avg_computed_rb`) that are fairly close to the respective raw averages, implying that the original averages align well with user trends.

4. Rating Counts
   - The median `nbr_ratings_ba` (3) and `nbr_ratings_rb` (5) are relatively low, showing that many beers have few ratings on both sites, though RateBeer has slightly more ratings per beer on average.
   - RateBeer’s broader user base also shows a higher standard deviation in the count of matched valid ratings (`nbr_matched_valid_ratings_rb` is 80.10 vs. 42.51 for BeerAdvocate), meaning some beers have far more reviews than others on RateBeer.

5. Z-scores (Standardized Ratings)
   - The mean `zscore_ba` (-0.41) is lower than `zscore_rb` (-0.10), suggesting BeerAdvocate users' ratings tend to deviate more below the mean, which may imply a slight rating inflation on RateBeer.
   - BeerAdvocate’s z-scores also exhibit a broader spread, evidenced by a standard deviation of 0.81 compared to RateBeer's 0.73, reinforcing the idea that BeerAdvocate reviews show more polarized opinions.

6. Score Difference and Similarity
   - `diff_scores` has a mean of 0.798, which indicates a considerable average difference in the ratings across the two platforms.
   - `sim_scores`, however, is high (mean of 0.986), suggesting that while individual score values differ, the rating trends between matched pairs are generally consistent.

##### Breweries dataset

Now we can take a look at our breweries dataframe. For each platform, every brewery has a location, a name, and the amount of beers they produce, along with a unique identifier. We also have the similarity and difference scores between the breweries from the two platforms. The brewery location can be very useful for our research questions. We will look into this more later on in this notebook.

In [11]:
df_breweries.sample(5)

id_ba,location_ba,name_ba,nbr_beers_ba,id_rb,location_rb,name_rb,nbr_beers_rb,diff_scores,sim_scores
i64,str,str,i64,i64,str,str,i64,f64,f64
18188,"""England""","""Dunham Massey Brewing Company""",17,9067,"""England""","""Dunham Massey""",28,0.365064,0.967313
5707,"""Germany""","""Airbräu""",8,860,"""Germany""","""Airbräu""",11,1.0,1.0
6394,"""England""","""Alcazar Brewing Co.""",11,3659,"""England""","""Alcazar""",39,0.474283,0.903298
36697,"""Brazil""","""Morada Cia Etílica""",11,13081,"""Brazil""","""Morada Cia Etílica""",10,0.672809,1.0
34614,"""Scotland""","""Glenfinnan Brewery""",0,8523,"""Scotland""","""Glenfinnan""",3,0.662249,0.95591


The dataset has the following structure

| Column Name | Description |
|-------------|-------------|
| `id_ba` / `id_rb` | Unique identifier for each brewery in BA/RB | 
| `name_ba` / `name_rb` | Name of the brewery in BA/RB |
| `location_ba` / `location_rb` | Geographic location of the brewery in BA/RB |
| `nbr_beers_ba` / `nbr_beers_rb` | Number of beers produced by the brewery in BA/RB |
| `diff_scores` | Difference in scores between BA and RB |
| `sim_scores` | Similarity in scores between BA and RB |

In [12]:
describe(df_breweries)

+--------------+---------+------------------+---------------+-----------+-----------------------+-------------------+
| Column       | Type    |   Not null count |   Nulls count |   Nulls % |   Unique values count |   Unique values % |
|--------------+---------+------------------+---------------+-----------+-----------------------+-------------------|
| id_ba        | Int64   |             8281 |             0 |    0.00 % |                  8281 |          100.00 % |
| location_ba  | String  |             8281 |             0 |    0.00 % |                   205 |            2.48 % |
| name_ba      | String  |             8281 |             0 |    0.00 % |                  8281 |          100.00 % |
| nbr_beers_ba | Int64   |             8281 |             0 |    0.00 % |                   231 |            2.79 % |
| id_rb        | Int64   |             8281 |             0 |    0.00 % |                  8235 |           99.44 % |
| location_rb  | String  |             8281 |           

In [13]:
describe(df_breweries, filters=["id_ba", "id_rb"])

+--------------+---------+------------------+---------------+-----------+-----------------------+-------------------+
| Column       | Type    |   Not null count |   Nulls count |   Nulls % |   Unique values count |   Unique values % |
|--------------+---------+------------------+---------------+-----------+-----------------------+-------------------|
| location_ba  | String  |             8281 |             0 |    0.00 % |                   205 |            2.48 % |
| name_ba      | String  |             8281 |             0 |    0.00 % |                  8281 |          100.00 % |
| nbr_beers_ba | Int64   |             8281 |             0 |    0.00 % |                   231 |            2.79 % |
| location_rb  | String  |             8281 |             0 |    0.00 % |                   205 |            2.48 % |
| name_rb      | String  |             8281 |             0 |    0.00 % |                  8235 |           99.44 % |
| nbr_beers_rb | Int64   |             8281 |           

For our breweries on both platforms, we have no missing values (in this matched data).

##### Users dataset

**Note**: The users_approx data is essentailly exactly the same however rather than users being matched exactly by username across the two platforms, they are matched using a TF-IDF vectorizer and cosine similarity. Users are matched if username is close enough and same location.

For each of our users (for each platform), we have their number of ratings/reviews, their ID, their name, the timestamp of when they joined and their location. The location is interesting for us, along with the number of reviews of each user.

In [14]:
df_users.sample(5)

joined_ba,location_ba,nbr_ratings_ba,nbr_reviews_ba,user_id_ba,user_name_ba,user_name_lower_ba,joined_rb,location_rb,nbr_ratings_rb,user_id_rb,user_name_rb,user_name_lower_rb
f64,str,i64,i64,str,str,str,f64,str,i64,i64,str,str
1.3016e9,"""United States, Wisconsin""",20,20,"""scones.587710""","""scones""","""scones""",1.3022e9,"""United States, Wisconsin""",7,126563,"""scones""","""scones"""
1.4030e9,"""United States, Oklahoma""",112,3,"""lawsonciv.807685""","""lawsonciv""","""lawsonciv""",1.2549e9,"""United States, Oklahoma""",23,96278,"""Lawsonciv""","""lawsonciv"""
1.3271e9,"""United States, Illinois""",1176,107,"""docrock.655960""","""Docrock""","""docrock""",1.3888e9,"""United States, Illinois""",3,295871,"""DocRock""","""docrock"""
1.3804e9,"""United States, North Carolina""",6,0,"""gurn13.756292""","""Gurn13""","""gurn13""",1.3795e9,"""United States, North Carolina""",3,279873,"""gurn13""","""gurn13"""
1.1201e9,"""United States, Oregon""",7,7,"""kevinphish.26808""","""kevinphish""","""kevinphish""",1.1398e9,"""United States, Oregon""",23,33493,"""kevinphish""","""kevinphish"""


The dataset has the following structure
| Column Name | Description |
|-------------|-------------|
| `user_id_ba` / `user_id_rb` | Unique identifier for each user in BA/RB |
| `user_name_ba` / `user_name_rb` | Username of the reviewer in BA/RB |
| `user_name_lower_ba` / `user_name_lower_rb` | Lowercase username in BA/RB |
| `joined_ba` / `joined_rb` | Date when the user joined BA/RB |
| `location_ba` / `location_rb` | Geographic location of the user in BA/RB |
| `nbr_ratings_ba` / `nbr_ratings_rb` | Number of ratings submitted by the user in BA/RB |
| `nbr_reviews_ba` | Number of reviews submitted by the user in BA |

In [15]:
describe(df_users)

+--------------------+---------+------------------+---------------+-----------+-----------------------+-------------------+
| Column             | Type    |   Not null count |   Nulls count |   Nulls % |   Unique values count |   Unique values % |
|--------------------+---------+------------------+---------------+-----------+-----------------------+-------------------|
| joined_ba          | Float64 |             3020 |             0 |    0.00 % |                  2205 |           73.01 % |
| location_ba        | String  |             3020 |             0 |    0.00 % |                   110 |            3.64 % |
| nbr_ratings_ba     | Int64   |             3020 |             0 |    0.00 % |                   629 |           20.83 % |
| nbr_reviews_ba     | Int64   |             3020 |             0 |    0.00 % |                   461 |           15.26 % |
| user_id_ba         | String  |             3020 |             0 |    0.00 % |                  3020 |          100.00 % |
| user_n

In [16]:
describe_numbers(df_users, filters=["user_id_ba", "user_id_rb"])

+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
| Column         |        Mean |         Std |         25% |         50% |         75% |         Min |         Max |
|----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------|
| joined_ba      | 1.27904e+09 | 1.09301e+08 | 1.20316e+09 | 1.28092e+09 | 1.36878e+09 | 9.81112e+08 |  1.5015e+09 |
| nbr_ratings_ba |     219.902 |     712.113 |           2 |          13 |         112 |           1 |       12046 |
| nbr_reviews_ba |     125.312 |     471.773 |           1 |           3 |          31 |           0 |        7593 |
| joined_rb      | 1.28179e+09 | 1.09276e+08 | 1.20376e+09 | 1.29604e+09 | 1.36369e+09 |  9.5683e+08 | 1.50037e+09 |
| nbr_ratings_rb |     285.698 |     1186.46 |           2 |           5 |          42 |           1 |       20678 |
+----------------+-------------+-------------+-------------+----

To make the `joined` columns more readable we cast them to a datetime object.

In [17]:
df_users = df_users.with_columns((pl.col("joined_ba").cast(pl.Int64) * 1000).cast(pl.Datetime("ms")))
df_users = df_users.with_columns((pl.col("joined_rb").cast(pl.Int64) * 1000).cast(pl.Datetime("ms")))
df_users.sample(5)

joined_ba,location_ba,nbr_ratings_ba,nbr_reviews_ba,user_id_ba,user_name_ba,user_name_lower_ba,joined_rb,location_rb,nbr_ratings_rb,user_id_rb,user_name_rb,user_name_lower_rb
datetime[ms],str,i64,i64,str,str,str,datetime[ms],str,i64,i64,str,str
2013-06-19 10:00:00,"""Spain""",83,0,"""torriman.737815""","""torriman""","""torriman""",2013-06-22 10:00:00,"""Spain""",21,265436,"""torriman""","""torriman"""
2010-03-20 11:00:00,"""United States, Virginia""",3,0,"""sasmall.440116""","""sasmall""","""sasmall""",2010-06-29 10:00:00,"""United States, Virginia""",3,109007,"""sasmall""","""sasmall"""
2009-06-05 10:00:00,"""United States, Michigan""",1,1,"""meadowswine.337218""","""Meadowswine""","""meadowswine""",2008-08-16 10:00:00,"""United States, Michigan""",3,80329,"""Meadowswine""","""meadowswine"""
2003-09-22 10:00:00,"""United States, Colorado""",1392,1392,"""crosling.3023""","""Crosling""","""crosling""",2004-09-28 10:00:00,"""United States, Colorado""",1539,15719,"""Crosling""","""crosling"""
2011-03-26 11:00:00,"""United States, California""",2,0,"""niekon.585329""","""Niekon""","""niekon""",2014-02-03 11:00:00,"""United States, California""",8,301012,"""Niekon""","""niekon"""


##### Ratings

In [18]:
df_ratings.sample(5)

abv_ba,appearance_ba,aroma_ba,beer_id_ba,beer_name_ba,brewery_id_ba,brewery_name_ba,date_ba,overall_ba,palate_ba,rating_ba,review_ba,style_ba,taste_ba,text_ba,user_id_ba,user_name_ba,abv_rb,appearance_rb,aroma_rb,beer_id_rb,beer_name_rb,brewery_id_rb,brewery_name_rb,date_rb,overall_rb,palate_rb,rating_rb,style_rb,taste_rb,text_rb,user_id_rb,user_name_rb
f64,f64,f64,i64,str,i64,str,i64,f64,f64,f64,bool,str,f64,str,str,str,f64,f64,f64,i64,str,i64,str,i64,f64,f64,f64,str,f64,str,i64,str
8.0,4.75,4.75,277238,"""Unique Friendshipz""",41018,"""The Veil Brewing Co.""",1496311200,4.5,4.75,4.6,true,"""American Double / Imperial IPA""",4.5,"""Sample at Mikkeller Beer Celeb…","""ciocanelu.691982""","""Ciocanelu""",8.0,5.0,9.0,507188,"""The Veil Unique Friendshipz""",26864,"""The Veil Brewing Company""",1496397600,18.0,4.0,4.5,"""Imperial IPA""",9.0,"""Sample at Mikkeller Beer Celeb…",274310,"""ciocanelu"""
7.6,4.0,4.0,91467,"""Sigtuna Dark Easter Ale""",14439,"""Sigtuna Brygghus""",1395572400,4.25,3.0,3.95,true,"""American Black Ale""",4.0,"""Bottle from Systembolaget. The…","""craftbeersweden.779415""","""craftbeersweden""",7.6,4.0,8.0,201942,"""Sigtuna Dark Easter Ale""",6774,"""Sigtuna Brygghus""",1364810400,17.0,3.0,4.0,"""American Strong Ale""",8.0,"""Bottle from Systembolaget. The…",133537,"""craftbeersweden"""
6.0,null,null,83894,"""Saison Rustique""",13905,"""Brasserie Dunham""",1368180000,null,null,3.75,true,"""Saison / Farmhouse Ale""",null,"""nan""","""whatup14.728898""","""Whatup14""",6.0,3.0,8.0,171337,"""Dunham Saison Rustique""",12142,"""Brasserie Dunham""",1368180000,17.0,3.0,3.8,"""Saison""",7.0,"""Bouteille 750ml. Couleur jaune…",188026,"""Whatup14"""
5.9,3.75,3.25,185836,"""Black And Wit""",35857,"""Hobbs Tavern & Brewing Co.""",1439287200,3.25,3.25,3.28,true,"""Witbier""",3.25,"""On tap at the source. Dark bro…","""johnnnniee.125029""","""johnnnniee""",5.9,3.0,6.0,353342,"""Hobbs Tavern Black and Wit""",20996,"""Hobbs Tavern & Brewing Company""",1439287200,14.0,3.0,3.2,"""Witbier""",6.0,"""On tap at the source. Dark bro…",60182,"""johnnnniee"""
6.75,4.5,4.5,20518,"""Sanctification""",863,"""Russian River Brewing Company""",1304589600,4.5,4.0,4.45,true,"""American Wild Ale""",4.5,"""Popped open a 375, pours a 2 f…","""jcartamdg.533452""","""jcartamdg""",6.75,4.0,8.0,38052,"""Russian River Sanctification""",1480,"""Russian River Brewing""",1337594400,19.0,4.0,4.4,"""Sour/Wild Ale""",9.0,"""Popped open a 375, pours a 2 f…",127131,"""jcartamdg"""


In [19]:
describe(df_ratings)

+-----------------+---------+------------------+---------------+-----------+-----------------------+-------------------+
| Column          | Type    |   Not null count |   Nulls count |   Nulls % |   Unique values count |   Unique values % |
|-----------------+---------+------------------+---------------+-----------+-----------------------+-------------------|
| abv_ba          | Float64 |            21964 |             0 |    0.00 % |                   187 |            0.85 % |
| appearance_ba   | Float64 |            19295 |          2669 |   12.15 % |                    18 |            0.09 % |
| aroma_ba        | Float64 |            19295 |          2669 |   12.15 % |                    18 |            0.09 % |
| beer_id_ba      | Int64   |            21964 |             0 |    0.00 % |                  9025 |           41.09 % |
| beer_name_ba    | String  |            21964 |             0 |    0.00 % |                  8719 |           39.70 % |
| brewery_id_ba   | Int64   |   

The dataset has the following structure

| Column Name | Description |
|-------------|-------------|
| `beer_id_ba` / `beer_id_rb` | Unique identifier for the beer in BA/RB |
| `beer_name_ba` / `beer_name_rb` | Name of the beer in BA/RB |
| `brewery_id_ba` / `brewery_id_rb` | Unique identifier for the brewery in BA/RB |
| `brewery_name_ba` / `brewery_name_rb` | Name of the brewery in BA/RB |
| `style_ba` / `style_rb` | Beer style in BA/RB |
| `abv_ba` / `abv_rb` | Alcohol By Volume percentage in BA/RB |
| `user_id_ba` / `user_id_rb` | Unique identifier for the user in BA/RB |
| `user_name_ba` / `user_name_rb` | Username of the reviewer in BA/RB |
| `date_ba` / `date_rb` | Date of the review in BA/RB |
| `rating_ba` / `rating_rb` | Overall rating given by the user in BA/RB |
| `overall_ba` / `overall_rb` | Overall score in BA/RB |
| `appearance_ba` / `appearance_rb` | Appearance score in BA/RB |
| `aroma_ba` / `aroma_rb` | Aroma score in BA/RB |
| `palate_ba` / `palate_rb` | Palate score in BA/RB |
| `taste_ba` / `taste_rb` | Taste score in BA/RB |
| `text_ba` / `text_rb` | Review text in BA/RB |
| `review_ba` | Whether or not there is a BA review available |

In [20]:
describe_numbers(df_ratings, filters=["beer_id_ba", "beer_id_rb", "brewery_id_ba", "brewery_id_rb", "user_id_ba", "user_id_rb"])

+---------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
| Column        |        Mean |         Std |         25% |         50% |         75% |         Min |         Max |
|---------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------|
| abv_ba        |     6.80022 |     2.09619 |         5.2 |         6.4 |           8 |         0.5 |          39 |
| appearance_ba |     3.78329 |    0.562042 |         3.5 |           4 |           4 |           1 |           5 |
| aroma_ba      |     3.70404 |    0.597862 |         3.5 |        3.75 |           4 |           1 |           5 |
| date_ba       | 1.34367e+09 | 1.12069e+08 | 1.27098e+09 | 1.36835e+09 | 1.43471e+09 | 9.84136e+08 | 1.50141e+09 |
| overall_ba    |     3.73419 |    0.614882 |         3.5 |           4 |           4 |           1 |           5 |
| palate_ba     |     3.67871 |    0.618717 |         3.5 |        3.75 

Here we see:

1. Alcohol By Volume (ABV)
   - The `abv_ba` and `abv_rb` columns are identical again of course.

2. Rating Categories on BeerAdvocate (BA)
   - `appearance_ba`, `aroma_ba`, `palate_ba`, `taste_ba`, and `overall_ba` ratings on BeerAdvocate have similar means (ranging from 3.68 to 3.78) with a consistent standard deviation of around 0.6.
   - The median for each category is close to 4, suggesting that BeerAdvocate users tend to give positive ratings overall, as the maximum for these is 5.
   - The overall rating (`rating_ba`) has a mean of 3.71, indicating a generally favorable tendency in ratings across these categories.

3. Rating Categories on RateBeer (RB)
   - `appearance_rb`, `palate_rb`, and `rating_rb` have means similar to those on BeerAdvocate (3.53 to 3.69), suggesting that for these categories, user ratings are comparable across platforms.
   - `aroma_rb` and `taste_rb`, however, have higher maximum scores of 10, with means of 6.96 and 7.01, respectively. This difference likely reflects a different scoring system or metric on RateBeer, which will need to be accounted for in our analysis.
   - `overall_rb` shows a much broader scale, with a mean of 14.40 and a maximum of 20, indicating that RateBeer may use a larger rating scale for this category than BeerAdvocate does. Again this will need to be accounted for.

4. General Trends
   - BeerAdvocate ratings are generally more constrained within a 1-5 range for all categories, while RateBeer uses a broader scale for certain categories (`aroma_rb`, `taste_rb`, and `overall_rb`). This suggests a difference in rating systems that should be considered in cross-platform comparisons.
   - Despite these scale differences, the means for comparable categories like `appearance`, `palate`, and `rating` are fairly aligned between platforms, implying similar user perceptions on these aspects of beer.
